In [1]:
#load libraries
import pandas as pd
import os as os
import numpy as np
import re
import ipywidgets as widgets
from ipywidgets import interactive
pd.set_option('display.max_columns', 300)
pd.set_option('display.max_rows', 300)

## Crawled database from https://www.henleypassportindex.com

In [2]:
# scrapping script here: https://github.com/jeremielamboley/world-passport-power-rank-enriched/blob/master/henley_passport_index_data_collection.ipynb
df_visa_free = pd.read_csv('https://raw.githubusercontent.com/jeremielamboley/henley/master/henley_visa_free_11_01_2019.csv', encoding = "ISO-8859-1")

In [3]:
df_visa_free.rename(columns={'passport': 'passportfrom_alpha_2code'}, inplace=True)
df_visa_free.rename(columns={'to': 'countryto_alpha_2code'}, inplace=True)

In [4]:
df_visa_free.shape

(45173, 3)

In [5]:
df_visa_free['passportfrom_alpha_2code'].nunique()

198

In [6]:
df_visa_free['countryto_alpha_2code'].nunique()

226

In [7]:
# Let's consider the passport holders countries as a country which gives visa-free access
df_visa_free.loc[df_visa_free['passportfrom_alpha_2code'] == df_visa_free['countryto_alpha_2code'], 'visafree'] = 1

In [8]:
# visafree matrix preview
df_visa_free.sample(5)

,passportfrom_alpha_2code,countryto_alpha_2code,visafree
30793,IQ,HK,0
4927,TV,VG,1
34552,CL,BZ,1
6328,UZ,HU,0
8193,TN,HU,0


In [9]:
# Nationals have visa free access to their own countries
df_visa_free[df_visa_free.passportfrom_alpha_2code == df_visa_free.countryto_alpha_2code].head()

,passportfrom_alpha_2code,countryto_alpha_2code,visafree
9,AM,AM,1
234,AG,AG,1
267,AO,AO,1
316,AR,AR,1
351,DZ,DZ,1


## List of world countries with ISO codes
### Source Wikipedia https://en.wikipedia.org/wiki/List_of_ISO_3166_country_codes
### Project by Radcliff https://gist.github.com/radcliff/f09c0f88344a7fcef373

In [10]:
df_iso = pd.read_csv('https://gist.githubusercontent.com/radcliff/f09c0f88344a7fcef373/raw/2753c482ad091c54b1822288ad2e4811c021d8ec/wikipedia-iso-country-codes.csv', encoding = "utf8")

In [11]:
df_iso.rename(columns={'English short name lower case': 'Country name'}, inplace=True)
df_iso.rename(columns={'Alpha-2 code': 'country_alpha_2code'}, inplace=True)
df_iso.rename(columns={'Alpha-3 code': 'country_alpha_3code'}, inplace=True)
df_iso.rename(columns={'Numeric code': 'country_numeric_code'}, inplace=True)
df_iso = df_iso.drop(['ISO 3166-2'], axis=1)

In [12]:
df_iso.shape

(246, 4)

In [13]:
df_iso.sample(5)

,Country name,country_alpha_2code,country_alpha_3code,country_numeric_code
23,Benin,BJ,BEN,204
57,Czech Republic,CZ,CZE,203
165,Oman,OM,OMN,512
69,Falkland Islands (Malvinas),FK,FLK,238
0,Afghanistan,AF,AFG,4


## GDP per countries in USD, 2016, from the United Nations Statistics Division
### Source: http://data.un.org/

In [14]:
df_GDP_UN = pd.read_csv('http://data.un.org/_Docs/SYB/CSV/SYB61_T13_GDP%20and%20GDP%20Per%20Capita.csv', 
                        encoding = "ISO-8859-1", 
                        delimiter = ',',
                        header=1)

In [15]:
# filter the data to GDP in 2016
df_GDP_UN = df_GDP_UN[df_GDP_UN['Series'] == 'GDP in current prices (millions of US dollars)']
df_GDP_UN = df_GDP_UN[df_GDP_UN.Year == 2016]

In [16]:
# Organize the data
df_GDP_UN['Value'] = df_GDP_UN['Value'].str.replace(',', '')
df_GDP_UN['Value'] = df_GDP_UN['Value'].astype(int)
# Renaming columns
df_GDP_UN.rename(columns={'Region/Country/Area': 'country_numeric_code'}, inplace=True)
df_GDP_UN.rename(columns={'Value': 'UN_GDP_currentPrices2016'}, inplace=True)
# Convert GDP from millions to billions
df_GDP_UN['UN_GDP_currentPrices2016_billions'] = df_GDP_UN['UN_GDP_currentPrices2016'] / 1000

In [17]:
df_GDP_UN.shape

(242, 8)

In [18]:
df_GDP_UN.sample(5)

,country_numeric_code,Unnamed: 1,Year,Series,UN_GDP_currentPrices2016,Footnotes,Source,UN_GDP_currentPrices2016_billions
6084,780,Trinidad and Tobago,2016,GDP in current prices (millions of US dollars),24086,NaN,"United Nations Statistics Division, New York, ...",24.086
1198,44,Bahamas,2016,GDP in current prices (millions of US dollars),11262,NaN,"United Nations Statistics Division, New York, ...",11.262
1038,32,Argentina,2016,GDP in current prices (millions of US dollars),545866,NaN,"United Nations Statistics Division, New York, ...",545.866
6544,704,Viet Nam,2016,GDP in current prices (millions of US dollars),205276,NaN,"United Nations Statistics Division, New York, ...",205.276
4621,558,Nicaragua,2016,GDP in current prices (millions of US dollars),13230,NaN,"United Nations Statistics Division, New York, ...",13.230


In [19]:
df_GDP_UN = df_GDP_UN[['country_numeric_code','UN_GDP_currentPrices2016_billions']]

### World total GDP for 2016 to get a % of coverage per passport

In [20]:
# country code = 1 for the world aggregate
df_GDP_UN['UN_World_GDP_percent'] = df_GDP_UN['UN_GDP_currentPrices2016_billions'] / df_GDP_UN[df_GDP_UN['country_numeric_code'] == 1].iloc[0,1]

In [21]:
df_GDP_UN.sample(5)

,country_numeric_code,UN_GDP_currentPrices2016_billions,UN_World_GDP_percent
5201,659,0.910,0.000012
1974,344,320.912,0.004242
2676,231,70.315,0.000929
2573,226,10.678,0.000141
5628,706,1.318,0.000017


# Population and surface, 2015, from the United Nations Statistics Division

### Source: http://data.un.org/

In [22]:
df_population_surface_UN = pd.read_csv('http://data.un.org/_Docs/SYB/CSV/SYB61_T02_Population,%20Surface%20Area%20and%20Density.csv', 
                        encoding = "ISO-8859-1", 
                        delimiter = ',',
                        header=1)

In [23]:
# filter the data to GDP in 2015
df_population_surface_UN = df_population_surface_UN[(df_population_surface_UN['Series'] == 'Population mid-year estimates (millions)') | (df_population_surface_UN['Series'] == 'Surface area (thousand km2)')]
df_population_surface_UN = df_population_surface_UN[df_population_surface_UN['Year'] == 2015]

In [24]:
# Organize the data
df_population_surface_UN['Value'] = df_population_surface_UN['Value'].str.replace(',', '')
df_population_surface_UN['Value'] = df_population_surface_UN['Value'].astype(float)

In [25]:
df_population_surface_UN = df_population_surface_UN[['Region/Country/Area','Series','Value']]
df_population_surface_UN.rename(columns={'Region/Country/Area': 'country_numeric_code'}, inplace=True)

In [26]:
# pivot and un-pivot
df_population_surface_UN = df_population_surface_UN.pivot(index='country_numeric_code', columns='Series', values='Value')
df_population_surface_UN = pd.DataFrame(df_population_surface_UN.to_records())

In [27]:
# get the world total for population and surface
df_population_UN_world = df_population_surface_UN[(df_population_surface_UN.country_numeric_code == 1)].iloc[0,1]
df_surface_UN_world = df_population_surface_UN[(df_population_surface_UN.country_numeric_code == 1)].iloc[0,2]

In [28]:
# World Population mid-year estimates 7383 millions, 2015
# Source: United Nations Statistics Division, New York, World Population Prospects: The 2017 Revision, last accessed June 2017.
df_population_surface_UN['UN_population_Value_2015_percent'] = (df_population_surface_UN['Population mid-year estimates (millions)'] / df_population_UN_world)

In [29]:
df_population_surface_UN['Surface_area_convered_percent'] = (df_population_surface_UN['Surface area (thousand km2)'] / df_surface_UN_world)

In [30]:
df_population_surface_UN.sample(5)

,country_numeric_code,Population mid-year estimates (millions),Surface area (thousand km2),UN_population_Value_2015_percent,Surface_area_convered_percent
199,643,143.89,17098.0,0.019489,0.125571
74,184,0.02,0.0,0.000003,0.000000
136,417,5.87,200.0,0.000795,0.001469
219,710,55.29,1221.0,0.007489,0.008967
102,275,4.66,6.0,0.000631,0.000044


# Unesco World Heritage list 

### https://whc.unesco.org/en/syndication

In [31]:
# Fetch the data from the Unesco website
unesco = pd.read_excel('https://whc.unesco.org/en/list/xls/whc-sites-2018.xls', encoding="Windows-1252",header=0)
unesco = unesco.reindex(['name_en', 'iso_code'], axis="columns")

In [32]:
# check the shape
unesco.shape

(1092, 2)

In [33]:
# Some values are comma separated and need to be reformated
unesco.tail(5)

,name_en,iso_code
1087,Prehistoric Rock Art Sites in the Côa Valley a...,"pt,es"
1088,Uvs Nuur Basin,"ru,mn"
1089,Landscapes of Dauria,"ru,mn"
1090,Heritage of Mercury. Almadén and Idrija,"si,es"
1091,Mosi-oa-Tunya / Victoria Falls,"zm,zw"


In [34]:
# Split the values
unesco = pd.concat([unesco['name_en'], unesco['iso_code'].str.split(',', expand=True)], axis=1)

unesco2 = []
for i in range(0,11):
    temp = unesco.loc[:,['name_en',i]]
    temp.columns = ['name_en','countryto_alpha_2code']
    unesco2.append(temp)
unesco2 = pd.concat(unesco2, ignore_index=True)

# Remove empty values
unesco2 = unesco2.replace(to_replace='None', value=np.nan).dropna().reset_index(drop=True)
# preview
unesco2

,name_en,countryto_alpha_2code
0,Cultural Landscape and Archaeological Remains ...,af
1,Minaret and Archaeological Remains of Jam,af
2,Historic Centres of Berat and Gjirokastra,al
3,Butrint,al
4,Al Qal'a of Beni Hammad,dz
5,Tassili n'Ajjer,dz
6,M'Zab Valley,dz
7,Djémila,dz
8,Tipasa,dz
9,Timgad,dz


In [35]:
# create a pivot table
unesco_pivot = pd.pivot_table(unesco2,index=['countryto_alpha_2code'], aggfunc='count')
unesco_pivot = pd.DataFrame(unesco_pivot.to_records())
unesco_pivot = unesco_pivot.rename(columns = {'name_en' :'unesco_total_sites'})
unesco_pivot = unesco_pivot.sort_values('unesco_total_sites',ascending=False)

In [36]:
# Count the total of world unesco sites
unesco_total_sites = unesco_pivot['unesco_total_sites'].sum()
# create the % of sites located in each country
unesco_pivot['unesco_total_sites_percent'] = (unesco_pivot['unesco_total_sites'] / unesco_total_sites)

In [37]:
# change to uppercase
unesco_pivot['countryto_alpha_2code'] = unesco_pivot['countryto_alpha_2code'].str.upper()

In [38]:
unesco_pivot.sample(5)

,countryto_alpha_2code,unesco_total_sites,unesco_total_sites_percent
98,MH,1,0.000858
127,PW,1,0.000858
119,PA,5,0.004288
40,DK,10,0.008576
77,KE,7,0.006003


## Intersect df_visa_free and df_iso on 'country to'

In [39]:
# copy df_visa_free, rename the columns from centadata and hauntedhouse
df_visa_free2 = df_visa_free.copy()

In [40]:
df_visa_free2.rename(columns={'countryto_alpha_2code': 'country_alpha_2code'}, inplace=True)

In [41]:
# Intersect on 'countryto_alpha_2code'
df = pd.merge(df_visa_free2, df_iso, on=['country_alpha_2code'], how='left')

In [42]:
df.rename(columns={'country_alpha_2code': 'countryto_alpha_2code'}, inplace=True)
df.rename(columns={'Country name': 'countryto_Country name'}, inplace=True)
df.rename(columns={'country_alpha_3code': 'countryto_country_alpha_3code'}, inplace=True)
df.rename(columns={'country_numeric_code': 'countryto_country_numeric_code'}, inplace=True)

In [43]:
df['passportfrom_alpha_2code'].nunique()

198

In [44]:
df['countryto_alpha_2code'].nunique()

226

In [45]:
df.sample(10)

,passportfrom_alpha_2code,countryto_alpha_2code,visafree,countryto_Country name,countryto_country_alpha_3code,countryto_country_numeric_code
3212,UZ,DZ,0,Algeria,DZA,12.0
41083,AT,IN,0,India,IND,356.0
21334,ML,CN,0,China,CHN,156.0
12265,ZA,HU,0,Hungary,HUN,348.0
6665,TO,CG,0,Congo,COG,178.0
23498,MV,FJ,1,Fiji,FJI,242.0
9650,LK,CI,0,Côte d'Ivoire,CIV,384.0
26729,ME,KR,1,"Korea, Republic of",KOR,410.0
39556,GE,ZM,1,Zambia,ZMB,894.0
21990,ML,CW,0,NaN,NaN,NaN


## Intersect df_visa_free and df_iso on 'passport'

In [46]:
df_iso.rename(columns={'country_alpha_2code': 'passportfrom_alpha_2code'}, inplace=True)

In [47]:
# Intersect on 'passportfrom_alpha_2code'
df = pd.merge(df, df_iso, on=['passportfrom_alpha_2code'], how='left')

In [48]:
df_iso.rename(columns={'passportfrom_alpha_2code': 'country_alpha_2code'}, inplace=True)

In [49]:
df.rename(columns={'Country name': 'passportfrom_Country name'}, inplace=True)

In [50]:
df = df.drop(['country_alpha_3code'], axis=1)
df = df.drop(['country_numeric_code'], axis=1)

## Intersect df and df_GDP_UN

In [51]:
df_GDP_UN2 = df_GDP_UN.copy()
df_GDP_UN2.rename(columns={'country_numeric_code': 'countryto_country_numeric_code'}, inplace=True)

In [52]:
df2 = pd.merge(df, df_GDP_UN2, on=['countryto_country_numeric_code'], how='left')

In [53]:
df2.rename(columns={'UN_GDP_currentPrices2016_billions': 'countryto_UN_GDP_currentPrices2016'}, inplace=True)

In [54]:
df2.sample(5)

,passportfrom_alpha_2code,countryto_alpha_2code,visafree,countryto_Country name,countryto_country_alpha_3code,countryto_country_numeric_code,passportfrom_Country name,countryto_UN_GDP_currentPrices2016,UN_World_GDP_percent
42096,EG,SL,0,Sierra Leone,SLE,694.0,Egypt,3.675,0.000049
15075,SG,MH,1,Marshall Islands,MHL,584.0,Singapore,0.183,0.000002
7899,ZW,TN,0,Tunisia,TUN,788.0,Zimbabwe,41.704,0.000551
2704,YE,CM,0,Cameroon,CMR,120.0,Yemen,32.217,0.000426
43288,FJ,UZ,0,Uzbekistan,UZB,860.0,Fiji,67.779,0.000896


## Intersect df2 and df_population_UN

In [55]:
df_population_UN2 = df_population_surface_UN.copy()
df_population_UN2.rename(columns={'country_numeric_code': 'countryto_country_numeric_code'}, inplace=True)

In [56]:
df3 = pd.merge(df2, df_population_UN2, on=['countryto_country_numeric_code'], how='left')

In [57]:
#df3.rename(columns={'UN_population_Value_2015_millions': 'countryto_UN_population_Value_2015_millions'}, inplace=True)
df3.rename(columns={'Population mid-year estimates (millions)': 'countryto_UN_population_Value_2015_millions'}, inplace=True)
df3.rename(columns={'Surface area (thousand km2)': 'countryto_Surface area (thousand km2)'}, inplace=True)

In [58]:
df3.sample(5)

,passportfrom_alpha_2code,countryto_alpha_2code,visafree,countryto_Country name,countryto_country_alpha_3code,countryto_country_numeric_code,passportfrom_Country name,countryto_UN_GDP_currentPrices2016,UN_World_GDP_percent,countryto_UN_population_Value_2015_millions,countryto_Surface area (thousand km2),UN_population_Value_2015_percent,Surface_area_convered_percent
7657,CH,CY,1,Cyprus,CYP,196.0,Switzerland,20.046,0.000265,1.16,9.0,0.000157,0.000066
29126,GH,BR,0,Brazil,BRA,76.0,Ghana,1795.926,0.023740,205.96,8516.0,0.027896,0.062543
19360,SL,UZ,0,Uzbekistan,UZB,860.0,Sierra Leone,67.779,0.000896,30.98,449.0,0.004196,0.003298
4733,SM,ME,1,Montenegro,MNE,499.0,San Marino,4.374,0.000058,0.63,14.0,0.000085,0.000103
23642,LV,KM,1,Comoros,COM,174.0,Latvia,1.150,0.000015,0.78,2.0,0.000106,0.000015


## Intersect df3 and unesco_pivot

In [59]:
unesco_pivot2 = unesco_pivot.copy()

In [60]:
df4 = pd.merge(df3, unesco_pivot2, on=['countryto_alpha_2code'], how='left')

In [61]:
df4.sample(5)

,passportfrom_alpha_2code,countryto_alpha_2code,visafree,countryto_Country name,countryto_country_alpha_3code,countryto_country_numeric_code,passportfrom_Country name,countryto_UN_GDP_currentPrices2016,UN_World_GDP_percent,countryto_UN_population_Value_2015_millions,countryto_Surface area (thousand km2),UN_population_Value_2015_percent,Surface_area_convered_percent,unesco_total_sites,unesco_total_sites_percent
1293,AG,SB,1,Solomon Islands,SLB,90.0,Antigua and Barbuda,1.134,0.000015,0.59,29.0,0.000080,0.000213,1.0,0.000858
9277,SO,BJ,1,Benin,BEN,204.0,Somalia,8.894,0.000118,10.58,115.0,0.001433,0.000845,2.0,0.001715
30843,KE,RS,0,Serbia,SRB,688.0,Kenya,38.300,0.000506,8.85,88.0,0.001199,0.000646,5.0,0.004288
38505,BE,CN,0,China,CHN,156.0,Belgium,11218.281,0.148294,1397.03,9600.0,0.189222,0.070504,53.0,0.045455
11293,SD,LR,0,Liberia,LBR,430.0,Sudan,2.757,0.000036,4.50,111.0,0.000610,0.000815,NaN,NaN


## Reordering the columns

In [62]:
df4 = df4.reset_index(drop=True)
cols = df4.columns.tolist()
cols

['passportfrom_alpha_2code',
 'countryto_alpha_2code',
 'visafree',
 'countryto_Country name',
 'countryto_country_alpha_3code',
 'countryto_country_numeric_code',
 'passportfrom_Country name',
 'countryto_UN_GDP_currentPrices2016',
 'UN_World_GDP_percent',
 'countryto_UN_population_Value_2015_millions',
 'countryto_Surface area (thousand km2)',
 'UN_population_Value_2015_percent',
 'Surface_area_convered_percent',
 'unesco_total_sites',
 'unesco_total_sites_percent']

In [63]:
df4 = df4[['passportfrom_alpha_2code','passportfrom_Country name','countryto_alpha_2code','countryto_Country name','countryto_country_alpha_3code', 'countryto_country_numeric_code','countryto_UN_GDP_currentPrices2016','UN_World_GDP_percent', 'countryto_UN_population_Value_2015_millions','UN_population_Value_2015_percent','countryto_Surface area (thousand km2)','Surface_area_convered_percent','unesco_total_sites','unesco_total_sites_percent','visafree']]

In [64]:
df4['passportfrom_alpha_2code'].nunique()

198

In [65]:
df4.head(5)

,passportfrom_alpha_2code,passportfrom_Country name,countryto_alpha_2code,countryto_Country name,countryto_country_alpha_3code,countryto_country_numeric_code,countryto_UN_GDP_currentPrices2016,UN_World_GDP_percent,countryto_UN_population_Value_2015_millions,UN_population_Value_2015_percent,countryto_Surface area (thousand km2),Surface_area_convered_percent,unesco_total_sites,unesco_total_sites_percent,visafree
0,AM,Armenia,AF,Afghanistan,AFG,4.0,20.235,0.000267,33.74,0.004570,653.0,0.004796,2.0,0.001715,0
1,AM,Armenia,AL,Albania,ALB,8.0,11.864,0.000157,2.92,0.000396,29.0,0.000213,3.0,0.002573,1
2,AM,Armenia,DZ,Algeria,DZA,12.0,159.049,0.002102,39.87,0.005400,2382.0,0.017494,7.0,0.006003,0
3,AM,Armenia,AS,American Samoa,ASM,16.0,NaN,NaN,0.06,0.000008,0.0,0.000000,NaN,NaN,0
4,AM,Armenia,AD,Andorra,AND,20.0,2.858,0.000038,0.08,0.000011,0.0,0.000000,1.0,0.000858,0


## Pandas pivot_table

In [66]:
df_pivot = pd.pivot_table(df4[df4.visafree == 1],index='passportfrom_Country name',values=['countryto_UN_GDP_currentPrices2016','UN_World_GDP_percent','countryto_UN_population_Value_2015_millions','UN_population_Value_2015_percent','countryto_Surface area (thousand km2)','Surface_area_convered_percent','unesco_total_sites','unesco_total_sites_percent','visafree'], aggfunc=np.sum, margins=False,dropna=True)
df_pivot.sort_values('visafree', ascending=False)
df_pivot.head(5)

,Surface_area_convered_percent,UN_World_GDP_percent,UN_population_Value_2015_percent,countryto_Surface area (thousand km2),countryto_UN_GDP_currentPrices2016,countryto_UN_population_Value_2015_millions,unesco_total_sites,unesco_total_sites_percent,visafree
passportfrom_Country name,,,,,,,,,
Afghanistan,0.043228,0.006096,0.053354,5886.0,461.147,393.91,41.0,0.035163,31
Albania,0.235925,0.304711,0.269534,32124.0,23051.078,1989.97,640.0,0.548885,116
Algeria,0.126034,0.040480,0.132256,17161.0,3062.282,976.45,164.0,0.140652,51
Andorra,0.541803,0.758506,0.424193,73773.0,57380.122,3131.82,916.0,0.785592,170
Angola,0.123030,0.044224,0.153492,16752.0,3345.458,1133.23,141.0,0.120926,50


## Turn the pivot table in a Data frame

In [67]:
df_pivot_flattened = pd.DataFrame(df_pivot.to_records())
df_pivot_flattened.sample(5)

,passportfrom_Country name,Surface_area_convered_percent,UN_World_GDP_percent,UN_population_Value_2015_percent,countryto_Surface area (thousand km2),countryto_UN_GDP_currentPrices2016,countryto_UN_population_Value_2015_millions,unesco_total_sites,unesco_total_sites_percent,visafree
21,Bosnia and Herzegovina,0.434673,0.467833,0.472531,59186.0,35391.035,3488.70,729.0,0.625214,119
170,Switzerland,0.581616,0.762144,0.440487,79194.0,57655.361,3252.12,937.0,0.803602,186
128,Nicaragua,0.387054,0.364420,0.311801,52702.0,27567.988,2302.03,703.0,0.602916,129
10,Azerbaijan,0.257818,0.072796,0.174219,35105.0,5506.915,1286.26,219.0,0.187822,67
168,Swaziland,0.118895,0.079574,0.165167,16189.0,6019.701,1219.43,177.0,0.151801,73


In [68]:
df_pivot_flattened['Surface_Rank'] = df_pivot_flattened['countryto_Surface area (thousand km2)'].rank(method='min', ascending=False).astype(int)
df_pivot_flattened['GDP_Rank'] = df_pivot_flattened['countryto_UN_GDP_currentPrices2016'].rank(method='min', ascending=False).astype(int)
df_pivot_flattened['Population_Rank'] = df_pivot_flattened['countryto_UN_population_Value_2015_millions'].rank(method='min', ascending=False).astype(int)
df_pivot_flattened['visafree_countries_Rank'] = df_pivot_flattened['visafree'].rank(method='min', ascending=False).astype(int)
df_pivot_flattened['unesco_total_sites_Rank'] = df_pivot_flattened['unesco_total_sites'].rank(method='min', ascending=False).astype(int)

## Calculate the overall ranking

In [69]:
df_pivot_flattened['Overall_ranking_calculation'] = df_pivot_flattened[['visafree_countries_Rank','GDP_Rank','Surface_Rank','Population_Rank','unesco_total_sites_Rank']].sum(axis=1)

In [70]:
df_pivot_flattened['New_Overall_ranking'] = df_pivot_flattened['Overall_ranking_calculation'].rank(method='min', ascending=True).astype(int)

## Create a new column Gain/loss in ranking

In [71]:
df_pivot_flattened['gain-loss'] = df_pivot_flattened['visafree_countries_Rank'] - df_pivot_flattened['New_Overall_ranking']

In [72]:
df_pivot_flattened.sample(5)

,passportfrom_Country name,Surface_area_convered_percent,UN_World_GDP_percent,UN_population_Value_2015_percent,countryto_Surface area (thousand km2),countryto_UN_GDP_currentPrices2016,countryto_UN_population_Value_2015_millions,unesco_total_sites,unesco_total_sites_percent,visafree,Surface_Rank,GDP_Rank,Population_Rank,visafree_countries_Rank,unesco_total_sites_Rank,Overall_ranking_calculation,New_Overall_ranking,gain-loss
116,"Moldova, Republic of",0.344384,0.280404,0.262554,46892.0,21212.273,1938.44,684.0,0.586621,123,87,96,101,88,87,459,93,-5
93,Kyrgyzstan,0.269392,0.074655,0.200497,36681.0,5647.571,1480.27,229.0,0.196398,65,97,129,120,133,118,597,118,15
43,Cyprus,0.428886,0.484213,0.365034,58398.0,36630.163,2695.05,857.0,0.734991,174,63,57,65,39,49,273,55,-16
188,United States,0.512889,0.718008,0.388461,69836.0,54316.455,2868.01,880.0,0.754717,186,36,40,57,12,44,189,39,-27
33,Chad,0.132886,0.051743,0.176803,18094.0,3914.264,1305.34,136.0,0.116638,54,142,145,139,159,166,751,156,3


## Reorder the columns

In [73]:
df_pivot_flattened = df_pivot_flattened[['passportfrom_Country name',
                                         'New_Overall_ranking',
                                         'gain-loss',
                                         'visafree',
                                         'visafree_countries_Rank',
                                         'countryto_UN_GDP_currentPrices2016',
                                         'GDP_Rank',
                                         'UN_World_GDP_percent',
                                         'countryto_UN_population_Value_2015_millions',
                                         'Population_Rank',
                                         'UN_population_Value_2015_percent',
                                         'countryto_Surface area (thousand km2)',
                                         'Surface_Rank',
                                         'Surface_area_convered_percent',
                                         'unesco_total_sites',
                                         'unesco_total_sites_percent',
                                         'unesco_total_sites_Rank',
                                         'Overall_ranking_calculation']]

In [74]:
# Re-order and format the values
df_pivot_flattened.sort_values('New_Overall_ranking',ascending=True, inplace=True)
df_pivot_flattened = df_pivot_flattened.reset_index(drop=True)
df_pivot_flattened.style.format({
    'UN_World_GDP_percent': '{:.2%}'.format,
    'UN_population_Value_2015_percent': '{:.2%}'.format,
    'Surface_area_convered_percent': '{:.2%}'.format,
    'unesco_total_sites_percent': '{:.2%}'.format
    })

,passportfrom_Country name,New_Overall_ranking,gain-loss,visafree,visafree_countries_Rank,countryto_UN_GDP_currentPrices2016,GDP_Rank,UN_World_GDP_percent,countryto_UN_population_Value_2015_millions,Population_Rank,UN_population_Value_2015_percent,countryto_Surface area (thousand km2),Surface_Rank,Surface_area_convered_percent,unesco_total_sites,unesco_total_sites_percent,unesco_total_sites_Rank,Overall_ranking_calculation
0,Japan,1,0,191,1,69726.3,1,92.17%,5923.63,1,80.23%,86657,4,63.64%,1024,87.82%,1,8
1,Singapore,2,0,190,2,69078.8,2,91.32%,4752.05,3,64.36%,90972,2,66.81%,1009,86.54%,3,12
2,"Korea, Republic of",3,-1,190,2,61527.8,6,81.33%,4877.4,2,66.06%,100405,1,73.74%,1017,87.22%,2,13
3,France,4,0,189,4,57942.3,7,76.59%,3378.6,16,45.76%,80242,9,58.93%,951,81.56%,5,41
4,Germany,5,-1,189,4,57885.7,8,76.52%,3350.6,17,45.38%,81357,8,59.75%,951,81.56%,5,42
5,Denmark,6,0,188,6,57874.5,9,76.50%,3347.62,18,45.34%,79793,10,58.60%,946,81.13%,7,50
6,Finland,6,0,188,6,57874.5,9,76.50%,3347.62,18,45.34%,79793,10,58.60%,946,81.13%,7,50
7,Sweden,6,0,188,6,57874.5,9,76.50%,3347.62,18,45.34%,79793,10,58.60%,946,81.13%,7,50
8,Italy,6,0,188,6,57874.5,9,76.50%,3347.62,18,45.34%,79793,10,58.60%,946,81.13%,7,50
9,San Marino,10,36,169,46,68521.7,3,90.58%,4481.94,4,60.71%,82795,7,60.81%,959,82.25%,4,64


In [75]:
# Export to CSV
df_pivot_flattened.to_csv('ranking.csv')

## Biggest winners and losers

In [76]:
df_rankgainloss = df_pivot_flattened.sort_values('gain-loss', ascending=False, inplace=False)
df_rankgainloss.style.format({
    'UN_World_GDP_percent': '{:.2%}'.format,
    'UN_population_Value_2015_percent': '{:.2%}'.format,
    'Surface_area_convered_percent': '{:.2%}'.format,
    'unesco_total_sites_percent': '{:.2%}'.format
    })

,passportfrom_Country name,New_Overall_ranking,gain-loss,visafree,visafree_countries_Rank,countryto_UN_GDP_currentPrices2016,GDP_Rank,UN_World_GDP_percent,countryto_UN_population_Value_2015_millions,Population_Rank,UN_population_Value_2015_percent,countryto_Surface area (thousand km2),Surface_Rank,Surface_area_convered_percent,unesco_total_sites,unesco_total_sites_percent,unesco_total_sites_Rank,Overall_ranking_calculation
146,Nepal,147,41,41,188,4985.3,135,6.59%,2477.53,74,33.56%,14532,168,10.67%,136,11.66%,166,731
9,San Marino,10,36,169,46,68521.7,3,90.58%,4481.94,4,60.71%,82795,7,60.81%,959,82.25%,4,64
77,Belarus,78,34,78,112,20420.1,98,26.99%,3200.66,31,43.35%,61453,53,45.13%,365,31.30%,98,392
58,Bosnia and Herzegovina,59,32,119,91,35391,62,46.78%,3488.7,13,47.25%,59186,62,43.47%,729,62.52%,75,303
16,Brunei Darussalam,17,31,166,48,65251.4,4,86.26%,4237.93,5,57.40%,84069,6,61.74%,884,75.81%,41,104
132,Lao People's Democratic Republic,133,30,53,163,5381.59,132,7.11%,1492.19,119,20.21%,33574,108,24.66%,169,14.49%,137,659
69,Ecuador,70,29,94,99,23254.8,90,30.74%,3395.52,15,45.99%,63537,49,46.66%,384,32.93%,97,350
49,Serbia,49,27,130,76,41425.4,48,54.76%,3591.97,12,48.65%,67201,45,49.35%,793,68.01%,62,243
111,Mongolia,112,26,62,138,9630.91,115,12.73%,1669.41,112,22.61%,47231,86,34.69%,262,22.47%,111,562
75,Fiji,76,25,90,101,22121.7,94,29.24%,3272.89,24,44.33%,57127,68,41.96%,349,29.93%,99,386


# A little program to access the results easily

In [77]:
# create a new list
countrylist = list(set(df_pivot_flattened['passportfrom_Country name'].drop_duplicates().values.tolist()))
countrylist.sort()

In [78]:
# Select the passport holder's country
countryselected = widgets.Dropdown(
    options=countrylist,
    value='Hong Kong',
    description="Country:",
    disabled=False,
)

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        print ("Country selected: %s" % change['new'])

countryselected.observe(on_change)
display(countryselected)

Dropdown(description='Country:', index=74, options=('Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola', …

In [79]:
# Selected country in the drop down menu
countryselectedvisafree = df_pivot_flattened[df_pivot_flattened['passportfrom_Country name'] == countryselected.value]['visafree']
countryselectedvisafree = countryselectedvisafree.to_frame()
countryselectedvisafree = countryselectedvisafree.iloc[0]['visafree']
# the overall country rank
countryrank = df_pivot_flattened[df_pivot_flattened['passportfrom_Country name'] == countryselected.value]['New_Overall_ranking']
countryrank = countryrank.to_frame()
countryrank = countryrank.iloc[0]['New_Overall_ranking']
# the gain-loss compare the to Henley Passport Index 
gainloss = df_pivot_flattened[df_pivot_flattened['passportfrom_Country name'] == countryselected.value]['gain-loss']
gainloss = gainloss.to_frame()
gainloss = gainloss.iloc[0]['gain-loss']
# the rank regarding the number of countries accessible visa-free
visafreecountriesRank = df_pivot_flattened[df_pivot_flattened['passportfrom_Country name'] == countryselected.value]['visafree_countries_Rank']
visafreecountriesRank = visafreecountriesRank.to_frame()
visafreecountriesRank = visafreecountriesRank.iloc[0]['visafree_countries_Rank']
# the rank regarding to the population accessible visa-free
countryselectedpopulation = df_pivot_flattened[df_pivot_flattened['passportfrom_Country name'] == countryselected.value]['UN_population_Value_2015_percent']
countryselectedpopulation = countryselectedpopulation.to_frame()
countryselectedpopulation = countryselectedpopulation.iloc[0]['UN_population_Value_2015_percent']
# the rank regarding to the globe surface accessible visa-free
countryselectedsurface = df_pivot_flattened[df_pivot_flattened['passportfrom_Country name'] == countryselected.value]['Surface_area_convered_percent']
countryselectedsurface = countryselectedsurface.to_frame()
countryselectedsurface = countryselectedsurface.iloc[0]['Surface_area_convered_percent']
countryselectedsurface = countryselectedsurface.item()
# the rank regarding to the world GDP accessible visa-free
countryselectedGDP = df_pivot_flattened[df_pivot_flattened['passportfrom_Country name'] == countryselected.value]['UN_World_GDP_percent']
countryselectedGDP = countryselectedGDP.to_frame()
countryselectedGDP = countryselectedGDP.iloc[0]['UN_World_GDP_percent']
# the rank regarding to the number of unesco sites accessible visa-free
countryselectedUnesco = df_pivot_flattened[df_pivot_flattened['passportfrom_Country name'] == countryselected.value]['unesco_total_sites_percent']
countryselectedUnesco = countryselectedUnesco.to_frame()
countryselectedUnesco = countryselectedUnesco.iloc[0]['unesco_total_sites_percent']

In [80]:
# Run this query to get the statistics
def country_access(countryselected,countryselectedvisafree,countryrank,visafreecountriesRank,gainloss,countryselectedpopulation,countryselectedsurface,countryselectedGDP):
    print("With " + countryselected.value + "'s passport you have access to:")
    print("\n" + str(countryselectedvisafree) + " countries visa-free, ranking " + str(countryrank)  +  " over " + str(df_pivot_flattened['passportfrom_Country name'].nunique()) + " countries' passports according to our new enriched ranking.")
    print("Originally ranked " + str(visafreecountriesRank) + " th by the Henley Passport Index. " + str(np.where(gainloss < 0, "Rank loss: "+str(gainloss), "Rank gain: +" + str(gainloss))))
    print("\nIt gives you visa-free access to:")
    print("\t" + "{:.0%}".format(countryselectedpopulation) + " of the world's population")
    print("\t" + "{:.0%}".format(countryselectedsurface) + " of the world's surface")
    print("\t" + "{:.0%}".format(countryselectedGDP) + " of the world's GPD")
    print("\t" + "{:.0%}".format(countryselectedUnesco) + " of the world's Unesco World heritage sites")

country_access(countryselected,countryselectedvisafree,countryrank,visafreecountriesRank,gainloss,countryselectedpopulation,countryselectedsurface,countryselectedGDP)

With Hong Kong's passport you have access to:

170 countries visa-free, ranking 34 over 197 countries' passports according to our new enriched ranking.
Originally ranked 42 th by the Henley Passport Index. Rank gain: +8

It gives you visa-free access to:
	40% of the world's population
	65% of the world's surface
	53% of the world's GPD
	79% of the world's Unesco World heritage sites
